<h2 align="center">BERT tutorial: Classify spam vs no spam emails</h2>

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

<h4>Import the dataset (Dataset is taken from kaggle)</h4>

In [2]:
import pandas as pd

df=pd.read_excel('../data/Constraint_Train.xlsx')
df.head(5)

,id,tweet,label
0,1.0,The CDC currently reports 99031 deaths. In gen...,real
1,2.0,States reported 1121 deaths a small rise from ...,real
2,3.0,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4.0,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5.0,Populous states can generate large case counts...,real


In [3]:
#df.groupby('Category').describe()

In [4]:
#df['Category'].value_counts()

In [5]:
#747/4825

**15% spam emails, 85% ham emails: This indicates class imbalance**

In [6]:
df_fake = df[df['label']=='fake']
df_fake.shape

(3060, 3)

In [7]:
df_real = df[df['label']=='real']
df_real.shape

(3360, 3)

In [8]:
df_real_downsampled = df_real.sample(df_fake.shape[0])
df_real_downsampled.shape

(3060, 3)

In [9]:
df_balanced = pd.concat([df_real_downsampled, df_fake])
df_balanced.shape

(6120, 3)

In [10]:
df_balanced['label'].value_counts()

real    3060
fake    3060
Name: label, dtype: int64

In [11]:
df_balanced['fake']=df_balanced['label'].apply(lambda x: 1 if x=='fake' else 0)
df_balanced.sample(5)

,id,tweet,label,fake
1778,1779.0,Arizona is the first U.S. state to begin repor...,real,0
251,252.0,Mar-a-Lago Tests Positive For the Coronavirus ...,fake,1
5623,5624.0,@InconvenientTr5 @AriFleischer @Cernovich Reme...,fake,1
2306,2307.0,Russia’s President Vladimir Putin claims that ...,fake,1
1175,1176.0,Nobel Laureate Dr. Honjo has claimed COVID-19 ...,fake,1


<h4>Split it into training and test data set</h4>

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_balanced['tweet'],df_balanced['fake'], stratify=df_balanced['fake'])

In [13]:
X_train.head(4)

465     Seen those pictures of dolphins and swans retu...
4274    The first case we are reporting today is a mal...
924     COVID 19 Situation in MH is "matter of concern...
4896    RT @HarvardGH: Contact tracing testing and sup...
Name: tweet, dtype: object

<h4>Now lets import BERT model and get embeding vectors for few sample statements</h4>

In [14]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [15]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.84351707, -0.5132726 , -0.8884571 , ..., -0.74748856,
        -0.7531474 ,  0.91964495],
       [-0.87208354, -0.50543964, -0.94446677, ..., -0.8584749 ,
        -0.7174535 ,  0.88082975]], dtype=float32)>

<h4>Get embeding vectors for few sample words. Compare them using cosine similarity</h4>

In [16]:
# e = get_sentence_embeding([
#     "banana", 
#     "grapes",
#     "mango",
#     "jeff bezos",
#     "elon musk",
#     "bill gates"
# ]
# )

In [17]:
# from sklearn.metrics.pairwise import cosine_similarity
# cosine_similarity([e[0]],[e[1]])

Values near to 1 means they are similar. 0 means they are very different.
Above you can use comparing "banana" vs "grapes" you get 0.99 similarity as they both are fruits

In [18]:
# cosine_similarity([e[0]],[e[3]])

Comparing banana with jeff bezos you still get 0.84 but it is not as close as 0.99 that we got with grapes

In [19]:
# cosine_similarity([e[3]],[e[4]])

Jeff bezos and Elon musk are more similar then Jeff bezos and banana as indicated above

<h4>Build Model</h4>

There are two types of models you can build in tensorflow. 

(1) Sequential
(2) Functional

So far we have built sequential model. But below we will build functional model. More information on these two is here: https://becominghuman.ai/sequential-vs-functional-model-in-keras-20684f766057

In [20]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

https://stackoverflow.com/questions/47605558/importerror-failed-to-import-pydot-you-must-install-pydot-and-graphviz-for-py

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [22]:
len(X_train)

4590

In [23]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

<h4>Train the model</h4>

In [24]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
 12/144 [=>............................] - ETA: 19:22 - loss: 0.7208 - accuracy: 0.4922 - precision: 0.4611 - recall: 0.4231

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\anite\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\anite\AppData\Local\Temp/ipykernel_7936/3696011659.py", line 1, in <module>
    model.fit(X_train, y_train, epochs=10)
  File "C:\Users\anite\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 64, in error_handler
    return fn(*args, **kwargs)
  File "C:\Users\anite\anaconda3\lib\site-packages\keras\engine\training.py", line 1384, in fit
    tmp_logs = self.train_function(iterator)
  File "C:\Users\anite\anaconda3\lib\site-packages\tensorflow\python\util\traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "C:\Users\anite\anaconda3\lib\site-packages\tensorflow\python\eager\def_function.py", line 915, in __call__
    result = self._call(*args, **kwds)
  File "C:\Users\anite\anaconda3\lib\site-packages\tensorflow\python\eager\de

TypeError: object of type 'NoneType' has no len()

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [ ]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm 

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_predicted))

<h4>Inference</h4>

In [ ]:
reviews = [
    'Enter a chance to win $5000, hurry up, offer valid until march 31, 2021',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
model.predict(reviews)